# Databricks Data Engineering 101: Medallion Architecture

**Build Production-Ready Data Pipelines with Bronze, Silver & Gold Layers**

Welcome to the core of data engineering on Databricks! In this notebook, you'll learn to:

- 🥉 **Bronze Layer**: Ingest raw data with complete history
- 🥈 **Silver Layer**: Clean, validate, and standardize data
- 🥇 **Gold Layer**: Create business-ready analytics tables

## What is Medallion Architecture?

The Medallion Architecture organizes data into three progressive layers:

```
Raw Data → [Bronze] → [Silver] → [Gold] → Business Insights
           Raw         Clean      Analytics
```

### Benefits:
- **Incremental Refinement**: Each layer adds value
- **Data Quality**: Progressive validation and cleansing
- **Performance**: Optimized for different use cases
- **Flexibility**: Easy to add new sources or metrics

### Real-World Use Cases:
- **Finance**: Transaction processing and fraud detection
- **E-commerce**: Customer analytics and product performance
- **Healthcare**: Patient records and clinical analytics
- **IoT**: Sensor data processing and anomaly detection

---


## Setup & Configuration

First, let's set up our environment and create database schemas for each layer.


In [ ]:
# Import libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

print("✅ Libraries imported successfully!")
print(f"Spark version: {spark.version}")


In [ ]:
-- Create databases/schemas for each layer
-- 
-- FOR COMMUNITY EDITION (Free): Uses Hive metastore
-- FOR PAID WORKSPACES with Unity Catalog: Change to CREATE SCHEMA
--   Example: CREATE SCHEMA IF NOT EXISTS main.bronze

CREATE DATABASE IF NOT EXISTS bronze
COMMENT 'Raw data layer - ingested as-is from source systems';

CREATE DATABASE IF NOT EXISTS silver
COMMENT 'Cleaned and validated data layer';

CREATE DATABASE IF NOT EXISTS gold
COMMENT 'Business-ready analytics layer';

SHOW DATABASES;


In [ ]:
# Configuration - CSV Data Location
# The notebook will automatically detect if running from Git Repos
# If not, it will look for CSVs in the path below

# Default path for manually uploaded CSVs
DATA_PATH = "/FileStore/tables"

# Alternative paths (uncomment if needed):
# DATA_PATH = "/Volumes/catalog/schema/volume"  # Unity Catalog Volume
# DATA_PATH = "dbfs:/mnt/data"                  # Mounted storage

print(f"Default data path: {DATA_PATH}")
print("(Auto-detection will check for Git Repos first)\n")


### 🎯 Auto-Detect: Git Repo vs Manual Upload

Run this cell to automatically detect your data source and create Bronze tables:


In [ ]:
# AUTO-DETECT & LOAD DATA
import os

print("="*70)
print("BRONZE LAYER: DATA INGESTION")
print("="*70)

# Try to detect data source
current_path = os.getcwd()
data_source = None
csv_path = None

# Option 1: Check if running from Git Repos
if "/Workspace/Repos/" in current_path:
    repo_data_path = f"{current_path}/data"
    try:
        # Try to access files in repo
        dbutils.fs.ls(f"file://{repo_data_path}")
        data_source = "git_repo"
        csv_path = f"file://{repo_data_path}"
        print(f"\n🎯 GIT REPO DETECTED")
        print(f"   Loading CSVs from: {repo_data_path}")
    except:
        pass

# Option 2: Use configured DATA_PATH (manual upload)
if not data_source:
    try:
        files = dbutils.fs.ls(DATA_PATH)
        csv_files = [f for f in files if f.name.endswith('.csv')]
        if csv_files:
            data_source = "manual_upload"
            csv_path = DATA_PATH
            print(f"\n📤 MANUAL UPLOAD DETECTED")
            print(f"   Loading CSVs from: {DATA_PATH}")
            print(f"   Found {len(csv_files)} CSV files")
    except Exception as e:
        print(f"\n⚠️  ERROR: Could not find CSV files!")
        print(f"   Tried: {DATA_PATH}")
        print(f"\n💡 To upload CSV files:")
        print(f"   1. Data → Create Table → Upload File")
        print(f"   2. Upload: customers.csv, products.csv, orders.csv, order_items.csv")
        print(f"   3. Update DATA_PATH variable above if needed")
        raise Exception("No data source found")

print(f"\n📋 Creating Bronze tables with PySpark...")
print("="*70)

# Define tables to create
tables = {
    "customers": "bronze.customers",
    "products": "bronze.products", 
    "orders": "bronze.orders",
    "order_items": "bronze.order_items"
}

# Load each CSV and create Bronze table
for csv_name, table_name in tables.items():
    print(f"\n⏳ Processing {csv_name}.csv...")
    
    # Read CSV
    csv_file = f"{csv_path}/{csv_name}.csv"
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    
    # Add metadata columns
    df = df.withColumn("_ingestion_timestamp", current_timestamp()) \
           .withColumn("_source_file", lit(f"{csv_name}.csv"))
    
    # Write to Bronze Delta table
    df.write.mode("overwrite").format("delta").saveAsTable(table_name)
    
    # Verify
    count = spark.table(table_name).count()
    print(f"   ✅ {table_name}: {count:,} records")

print("\n" + "="*70)
print("🎉 BRONZE LAYER COMPLETE!")
print("="*70)
print("\nAll CSV data has been loaded into Delta tables.")
print("You can now proceed to the Silver layer below.\n")


---

# 🥉 Bronze Layer: Verify & Explore

The Bronze tables have been created above! Let's verify and explore the data.


### Verify Bronze Tables


In [ ]:
-- Show all Bronze tables
SHOW TABLES IN bronze;


In [ ]:
-- Check record counts
SELECT 'customers' as table_name, COUNT(*) as record_count FROM bronze.customers
UNION ALL
SELECT 'products', COUNT(*) FROM bronze.products
UNION ALL
SELECT 'orders', COUNT(*) FROM bronze.orders
UNION ALL
SELECT 'order_items', COUNT(*) FROM bronze.order_items;


In [ ]:
-- Preview customers data
SELECT * FROM bronze.customers LIMIT 5;


### Preview Other Tables


In [ ]:
-- Preview products
SELECT * FROM bronze.products LIMIT 5;


### Data Quality Check


In [ ]:
-- Check for any null key columns (should be none)
SELECT
  'customers' as table_name,
  SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) as null_ids
FROM bronze.customers
UNION ALL
SELECT 
  'products',
  SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END)
FROM bronze.products
UNION ALL
SELECT 
  'orders',
  SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END)
FROM bronze.orders
UNION ALL
SELECT 
  'order_items',
  SUM(CASE WHEN line_item_id IS NULL THEN 1 ELSE 0 END)
FROM bronze.order_items;


### ✅ Bronze Layer Summary


In [ ]:
# Summary statistics
print("="*70)
print("BRONZE LAYER SUMMARY")
print("="*70)
print(f"\nCustomers:    {spark.table('bronze.customers').count():>10,} records")
print(f"Products:     {spark.table('bronze.products').count():>10,} records")
print(f"Orders:       {spark.table('bronze.orders').count():>10,} records")
print(f"Order Items:  {spark.table('bronze.order_items').count():>10,} records")

print("\n" + "="*70)
print("✅ Bronze Layer Complete - Raw data loaded into Delta tables")
print("="*70)
print("\nKey Features:")
print("  • All data stored in Delta Lake format (ACID compliant)")
print("  • Metadata columns added (_ingestion_timestamp, _source_file)")
print("  • Ready for cleansing in Silver layer")
print("\n")


---

# 🥈 Silver Layer: Data Cleansing & Validation

## Goals:
- **Clean**: Remove nulls, fix data types, standardize formats
- **Validate**: Apply business rules and constraints
- **Deduplicate**: Keep only unique, valid records
- **Enrich**: Add derived columns for downstream use

## Key Patterns:
- Data quality checks
- Deduplication using window functions
- Type casting and formatting
- Business rule validation


### Silver: Customers (Cleaned)


In [ ]:
-- Create Silver customers with data quality rules
DROP TABLE IF EXISTS silver.customers;

CREATE TABLE silver.customers USING DELTA AS
SELECT 
  customer_id,
  INITCAP(TRIM(first_name)) as first_name,  -- Standardize casing
  INITCAP(TRIM(last_name)) as last_name,
  LOWER(TRIM(email)) as email,              -- Lowercase emails
  phone,
  address,
  city,
  UPPER(state) as state,                    -- Uppercase state codes
  zip_code,
  country,
  registration_date,
  customer_segment,
  CURRENT_TIMESTAMP() as updated_at
FROM bronze.customers
WHERE 
  customer_id IS NOT NULL                   -- Must have ID
  AND email IS NOT NULL                     -- Must have email
  AND email LIKE '%@%'                      -- Valid email format
  AND registration_date IS NOT NULL;        -- Must have registration date

SELECT COUNT(*) as cleaned_count FROM silver.customers;


In [ ]:
-- Compare Bronze vs Silver
SELECT 
  'Bronze' as layer,
  COUNT(*) as record_count
FROM bronze.customers
UNION ALL
SELECT 
  'Silver' as layer,
  COUNT(*) as record_count
FROM silver.customers;


### Silver: Products (Cleaned)


In [ ]:
DROP TABLE IF EXISTS silver.products;

CREATE TABLE silver.products USING DELTA AS
SELECT 
  product_id,
  TRIM(product_name) as product_name,
  category,
  brand,
  price,
  stock_quantity,
  is_active,
  -- Add derived columns
  CASE 
    WHEN price < 50 THEN 'Budget'
    WHEN price < 200 THEN 'Mid-Range'
    ELSE 'Premium'
  END as price_tier,
  CURRENT_TIMESTAMP() as updated_at
FROM bronze.products
WHERE 
  product_id IS NOT NULL
  AND product_name IS NOT NULL
  AND price > 0                           -- Price must be positive
  AND price < 10000;                      -- Sanity check on max price

SELECT COUNT(*) as cleaned_count FROM silver.products;


### Silver: Orders (Cleaned & Enriched)


In [ ]:
DROP TABLE IF EXISTS silver.orders;

CREATE TABLE silver.orders USING DELTA AS
SELECT 
  order_id,
  customer_id,
  order_date,
  status,
  payment_method,
  shipped_date,
  delivered_date,
  discount_percent,
  -- Calculate fulfillment time
  CASE 
    WHEN delivered_date IS NOT NULL 
    THEN DATEDIFF(delivered_date, DATE(order_date))
    ELSE NULL
  END as days_to_deliver,
  -- Extract time dimensions
  YEAR(order_date) as order_year,
  MONTH(order_date) as order_month,
  DAYOFWEEK(order_date) as order_day_of_week,
  CURRENT_TIMESTAMP() as updated_at
FROM bronze.orders
WHERE 
  order_id IS NOT NULL
  AND customer_id IS NOT NULL
  AND order_date IS NOT NULL
  AND status IN ('Completed', 'Shipped', 'Processing', 'Cancelled');  -- Valid statuses only

SELECT COUNT(*) as cleaned_count FROM silver.orders;


### Silver: Order Items (Cleaned with Calculations)


In [ ]:
DROP TABLE IF EXISTS silver.order_items;

CREATE TABLE silver.order_items USING DELTA AS
SELECT 
  line_item_id,
  order_id,
  product_id,
  quantity,
  unit_price,
  -- Calculate line total
  quantity * unit_price as line_total,
  CURRENT_TIMESTAMP() as updated_at
FROM bronze.order_items
WHERE 
  line_item_id IS NOT NULL
  AND order_id IS NOT NULL
  AND product_id IS NOT NULL
  AND quantity > 0                        -- Positive quantity
  AND quantity <= 100                     -- Sanity check
  AND unit_price > 0                      -- Positive price
  AND unit_price < 10000;                 -- Sanity check

SELECT COUNT(*) as cleaned_count FROM silver.order_items;


### ✅ Silver Layer Complete!

Summary of our cleansed data:


In [ ]:
# Compare Bronze vs Silver record counts
print("="*70)
print("BRONZE → SILVER DATA QUALITY REPORT")
print("="*70)

tables = ['customers', 'products', 'orders', 'order_items']
for table in tables:
    bronze_count = spark.table(f'bronze.{table}').count()
    silver_count = spark.table(f'silver.{table}').count()
    rejected = bronze_count - silver_count
    rejection_rate = (rejected / bronze_count * 100) if bronze_count > 0 else 0
    
    print(f"\n{table.upper()}:")
    print(f"  Bronze: {bronze_count:>10,}")
    print(f"  Silver: {silver_count:>10,}")
    print(f"  Rejected: {rejected:>8,} ({rejection_rate:.2f}%)")

print("="*70)


---

# 🥇 Gold Layer: Business Analytics

## Goals:
- Create **business-ready** tables optimized for reporting
- Pre-calculate **metrics and KPIs**
- Denormalize data for **fast queries**
- Support **dashboards and analytics**

## Patterns:
- Aggregations and rollups
- Star schema / dimensional modeling
- Pre-calculated metrics
- Optimized for BI tools


### Gold: Customer Analytics

Calculate customer lifetime value and segmentation metrics.


In [ ]:
DROP TABLE IF EXISTS gold.customer_analytics;

CREATE TABLE gold.customer_analytics USING DELTA AS
WITH order_totals AS (
  SELECT 
    o.customer_id,
    o.order_id,
    SUM(oi.line_total) as order_total
  FROM silver.orders o
  INNER JOIN silver.order_items oi ON o.order_id = oi.order_id
  WHERE o.status != 'Cancelled'
  GROUP BY o.customer_id, o.order_id
)
SELECT 
  c.customer_id,
  c.first_name,
  c.last_name,
  c.email,
  c.city,
  c.state,
  c.customer_segment,
  c.registration_date,
  -- Order metrics
  COUNT(DISTINCT o.order_id) as total_orders,
  COALESCE(SUM(ot.order_total), 0) as lifetime_value,
  COALESCE(AVG(ot.order_total), 0) as avg_order_value,
  MAX(o.order_date) as last_order_date,
  MIN(o.order_date) as first_order_date,
  DATEDIFF(MAX(o.order_date), MIN(o.order_date)) as customer_tenure_days,
  -- Engagement score (orders per month as customer)
  CASE 
    WHEN DATEDIFF(MAX(o.order_date), MIN(o.order_date)) > 0 
    THEN COUNT(DISTINCT o.order_id) * 30.0 / DATEDIFF(MAX(o.order_date), MIN(o.order_date))
    ELSE 0
  END as orders_per_month,
  CURRENT_TIMESTAMP() as calculated_at
FROM silver.customers c
LEFT JOIN silver.orders o ON c.customer_id = o.customer_id AND o.status != 'Cancelled'
LEFT JOIN order_totals ot ON o.order_id = ot.order_id
GROUP BY 
  c.customer_id, c.first_name, c.last_name, c.email, 
  c.city, c.state, c.customer_segment, c.registration_date;

SELECT COUNT(*) as customer_count FROM gold.customer_analytics;


In [ ]:
-- Top 10 customers by lifetime value
SELECT 
  customer_id,
  first_name || ' ' || last_name as customer_name,
  email,
  total_orders,
  ROUND(lifetime_value, 2) as lifetime_value,
  ROUND(avg_order_value, 2) as avg_order_value,
  customer_segment
FROM gold.customer_analytics
ORDER BY lifetime_value DESC
LIMIT 10;


### Gold: Product Performance

Analyze product sales and revenue.


In [ ]:
DROP TABLE IF EXISTS gold.product_performance;

CREATE TABLE gold.product_performance USING DELTA AS
SELECT 
  p.product_id,
  p.product_name,
  p.category,
  p.brand,
  p.price,
  p.price_tier,
  -- Sales metrics
  COUNT(DISTINCT oi.order_id) as orders_containing_product,
  SUM(oi.quantity) as total_quantity_sold,
  SUM(oi.line_total) as total_revenue,
  AVG(oi.unit_price) as avg_selling_price,
  -- Rankings
  RANK() OVER (PARTITION BY p.category ORDER BY SUM(oi.line_total) DESC) as revenue_rank_in_category,
  CURRENT_TIMESTAMP() as calculated_at
FROM silver.products p
LEFT JOIN silver.order_items oi ON p.product_id = oi.product_id
LEFT JOIN silver.orders o ON oi.order_id = o.order_id AND o.status != 'Cancelled'
GROUP BY 
  p.product_id, p.product_name, p.category, p.brand, p.price, p.price_tier;

SELECT COUNT(*) as product_count FROM gold.product_performance;


In [ ]:
-- Top 10 products by revenue
SELECT 
  product_name,
  category,
  brand,
  ROUND(total_revenue, 2) as total_revenue,
  total_quantity_sold,
  orders_containing_product,
  revenue_rank_in_category
FROM gold.product_performance
ORDER BY total_revenue DESC
LIMIT 10;


### Gold: Monthly Revenue Trends

Time-series analysis for business reporting.


In [ ]:
DROP TABLE IF EXISTS gold.monthly_revenue;

CREATE TABLE gold.monthly_revenue USING DELTA AS
WITH order_revenues AS (
  SELECT 
    o.order_id,
    o.order_date,
    o.order_year,
    o.order_month,
    o.status,
    SUM(oi.line_total) as order_total,
    SUM(oi.line_total * o.discount_percent / 100) as discount_amount
  FROM silver.orders o
  INNER JOIN silver.order_items oi ON o.order_id = oi.order_id
  WHERE o.status != 'Cancelled'
  GROUP BY o.order_id, o.order_date, o.order_year, o.order_month, o.status
)
SELECT 
  order_year,
  order_month,
  DATE_TRUNC('month', order_date) as month_start_date,
  COUNT(DISTINCT order_id) as total_orders,
  SUM(order_total) as gross_revenue,
  SUM(discount_amount) as total_discounts,
  SUM(order_total - discount_amount) as net_revenue,
  AVG(order_total) as avg_order_value,
  -- Month-over-month growth
  LAG(SUM(order_total)) OVER (ORDER BY order_year, order_month) as prev_month_revenue,
  ROUND(
    (SUM(order_total) - LAG(SUM(order_total)) OVER (ORDER BY order_year, order_month)) 
    / LAG(SUM(order_total)) OVER (ORDER BY order_year, order_month) * 100, 
    2
  ) as mom_growth_percent,
  CURRENT_TIMESTAMP() as calculated_at
FROM order_revenues
GROUP BY order_year, order_month, DATE_TRUNC('month', order_date)
ORDER BY order_year, order_month;

SELECT COUNT(*) as month_count FROM gold.monthly_revenue;


In [ ]:
-- View monthly revenue trends
SELECT 
  TO_DATE(month_start_date) as month,
  total_orders,
  ROUND(gross_revenue, 2) as gross_revenue,
  ROUND(net_revenue, 2) as net_revenue,
  ROUND(avg_order_value, 2) as avg_order_value,
  mom_growth_percent
FROM gold.monthly_revenue
ORDER BY month DESC
LIMIT 12;


### Gold: Category Performance

Category-level analytics for merchandising decisions.


In [ ]:
DROP TABLE IF EXISTS gold.category_performance;

CREATE TABLE gold.category_performance USING DELTA AS
SELECT 
  p.category,
  COUNT(DISTINCT p.product_id) as total_products,
  COUNT(DISTINCT oi.order_id) as total_orders,
  SUM(oi.quantity) as total_units_sold,
  SUM(oi.line_total) as total_revenue,
  AVG(oi.line_total) as avg_transaction_value,
  MIN(p.price) as min_price,
  MAX(p.price) as max_price,
  AVG(p.price) as avg_price,
  CURRENT_TIMESTAMP() as calculated_at
FROM silver.products p
LEFT JOIN silver.order_items oi ON p.product_id = oi.product_id
LEFT JOIN silver.orders o ON oi.order_id = o.order_id AND o.status != 'Cancelled'
GROUP BY p.category;

SELECT * FROM gold.category_performance ORDER BY total_revenue DESC;


### ✅ Gold Layer Complete!

Summary of all Gold tables:


In [ ]:
SHOW TABLES IN gold;


In [ ]:
# Gold layer summary
print("="*70)
print("GOLD LAYER SUMMARY - BUSINESS ANALYTICS")
print("="*70)
print(f"\nCustomer Analytics:  {spark.table('gold.customer_analytics').count():>10,} customers")
print(f"Product Performance: {spark.table('gold.product_performance').count():>10,} products")
print(f"Monthly Revenue:     {spark.table('gold.monthly_revenue').count():>10,} months")
print(f"Category Performance:{spark.table('gold.category_performance').count():>10,} categories")
print("="*70)


---

# 🎓 Advanced Concepts

## Delta Lake Features You've Used

Throughout this notebook, you've leveraged powerful Delta Lake capabilities:


### Time Travel

Query previous versions of your data:


In [ ]:
-- View table history
DESCRIBE HISTORY silver.customers LIMIT 5;


In [ ]:
-- Query a previous version (if you've updated the table)
-- SELECT * FROM silver.customers VERSION AS OF 0;


### Table Statistics & Optimization


In [ ]:
-- View detailed table information
DESCRIBE EXTENDED gold.customer_analytics;


In [ ]:
-- Optimize tables for better query performance
-- OPTIMIZE gold.customer_analytics;
-- OPTIMIZE gold.product_performance;
-- OPTIMIZE gold.monthly_revenue;


---

# 🎉 Congratulations!

You've successfully built a complete **Medallion Architecture** on Databricks!

## What You Accomplished:

### ✅ Bronze Layer
- Ingested raw CSV data into Delta tables
- Preserved complete data history
- Used idempotent `COPY INTO` pattern

### ✅ Silver Layer
- Cleaned and validated data
- Applied business rules
- Added derived columns
- Standardized formats

### ✅ Gold Layer
- Created business-ready analytics tables
- Pre-calculated KPIs and metrics
- Built customer lifetime value analysis
- Analyzed product and category performance
- Created time-series revenue trends

## Key Concepts Mastered:

- 📦 **Delta Lake**: ACID transactions, time travel, schema evolution
- 🏗️ **Medallion Architecture**: Progressive data refinement
- 🔄 **ETL Patterns**: Incremental loading, data quality, transformations
- 📊 **Analytics Engineering**: Business metrics, aggregations, rankings
- 🚀 **Performance**: Optimizations, partitioning strategies

## Next Steps:

1. **Explore Further**: Try modifying queries to answer your own business questions
2. **Add Complexity**: Implement slowly changing dimensions (SCD Type 2)
3. **Automation**: Learn about Databricks Workflows to schedule these pipelines
4. **Streaming**: Explore Structured Streaming for real-time data
5. **ML Integration**: Build machine learning models on your clean data
6. **Check Best Practices**: Review notebook 03 for advanced patterns

---

## Sample Business Questions You Can Answer:

```sql
-- Who are the most valuable customers?
SELECT * FROM gold.customer_analytics 
ORDER BY lifetime_value DESC LIMIT 10;

-- What products drive the most revenue?
SELECT * FROM gold.product_performance 
ORDER BY total_revenue DESC LIMIT 10;

-- How is revenue trending?
SELECT * FROM gold.monthly_revenue 
ORDER BY order_year DESC, order_month DESC;

-- Which categories perform best?
SELECT * FROM gold.category_performance 
ORDER BY total_revenue DESC;
```

---

**You're now ready to build production data pipelines on Databricks! 🚀**

Questions? Check out:
- [Databricks Documentation](https://docs.databricks.com/)
- [Delta Lake Guide](https://docs.delta.io/)
- [Medallion Architecture](https://www.databricks.com/glossary/medallion-architecture)
